In [24]:
import pandas as pd
import numpy as np
import sys

In [25]:
adapters = pd.read_excel("C:\\Users\\Administrator\\Documents\\Coding\\DS\\adapters\\adapters_2711.xlsx")

In [26]:
adapters

,roche_p7,roche_p5_nextseq,china_i7,china_i5_no_nextseq,china_i5_nextseq
0,TGGATCGA,AGCTAGGT,TTGTGTCCTC,GTGATAAGCA,TGCTTATCAC
1,CAAGCTAG,GATCGAAC,CACCTCAGGA,TCCTATTAGG,CCTAATAGGA
2,GTACCAAG,GAACCATG,GGTACGTACT,CATGGCGTAC,GTACGCCATG
3,ACGTTGGA,AGGTTGCA,ACAGAAGTAG,AGACCGCCTT,AAGGCGGTCT
4,TCGTGGTA,ATGGTGCT,TAAGGAGTCA,CGTGCCGATT,AATCGGCACG
...,...,...,...,...,...
187,CGTTGTGA,AGTGTTGC,NaN,NaN,NaN
188,GCGACTAT,TATCAGCG,NaN,NaN,NaN
189,ATAGTCGC,CGCTGATA,NaN,NaN,NaN
190,GAGCTCAT,TACTCGAG,NaN,NaN,NaN


In [46]:
adapters['roche'] = adapters['roche_p7'] + adapters['roche_p5_nextseq']
adapters['china_nextseq'] = adapters.iloc[:96].apply(lambda row: row['china_i7'][:8] + row['china_i5_nextseq'][:8], axis=1)
adapters['china_no_nextseq'] = adapters.iloc[:96].apply(lambda row: row['china_i7'][:8] + row['china_i5_no_nextseq'][:8], axis=1)

In [48]:
adapters.head(99)

,roche_p7,roche_p5_nextseq,china_i7,china_i5_no_nextseq,china_i5_nextseq,roche,china_nextseq,china_no_nextseq
0,TGGATCGA,AGCTAGGT,TTGTGTCCTC,GTGATAAGCA,TGCTTATCAC,TGGATCGAAGCTAGGT,TTGTGTCCTGCTTATC,TTGTGTCCGTGATAAG
1,CAAGCTAG,GATCGAAC,CACCTCAGGA,TCCTATTAGG,CCTAATAGGA,CAAGCTAGGATCGAAC,CACCTCAGCCTAATAG,CACCTCAGTCCTATTA
2,GTACCAAG,GAACCATG,GGTACGTACT,CATGGCGTAC,GTACGCCATG,GTACCAAGGAACCATG,GGTACGTAGTACGCCA,GGTACGTACATGGCGT
3,ACGTTGGA,AGGTTGCA,ACAGAAGTAG,AGACCGCCTT,AAGGCGGTCT,ACGTTGGAAGGTTGCA,ACAGAAGTAAGGCGGT,ACAGAAGTAGACCGCC
4,TCGTGGTA,ATGGTGCT,TAAGGAGTCA,CGTGCCGATT,AATCGGCACG,TCGTGGTAATGGTGCT,TAAGGAGTAATCGGCA,TAAGGAGTCGTGCCGA
...,...,...,...,...,...,...,...,...
94,GTGCTTAG,GATTCGTG,GCCGAATTAA,CATTAGCTCT,AGAGCTAATG,GTGCTTAGGATTCGTG,GCCGAATTAGAGCTAA,GCCGAATTCATTAGCT
95,ACATCCGA,AGCCTACA,AGGACGCGTT,ACGGTCGGTA,TACCGACCGT,ACATCCGAAGCCTACA,AGGACGCGTACCGACC,AGGACGCGACGGTCGG
96,TGTACCTG,GTCCATGT,NaN,NaN,NaN,TGTACCTGGTCCATGT,NaN,NaN
97,CACGTTCA,ACTTGCAC,NaN,NaN,NaN,CACGTTCAACTTGCAC,NaN,NaN


In [49]:

def adapter_difference(str1,str2):

  result = 0

  #handle the case where one string is longer than the other
  maxlen = len(str2) if len(str1)<len(str2) else len(str1)

  #loop through the characters
  for i in range(maxlen):
    #use a slice rather than index in case one string longer than other
    letter1 = str1[i:i+1]
    letter2 = str2[i:i+1]
    #count difference
    if letter1 == letter2:
      result += 1

  #print out result
  return result

In [50]:
#Taking one adapter from the Chinese ones and comparing to every single one of Roche
def all_column_diff(adapter, adapters):
    diff_dict = {}
    for value in adapters['roche']:
        diff = adapter_difference(adapter, value)
        if diff in diff_dict:
            diff_dict[diff] += 1
        else:
            diff_dict[diff] = 1
    sorted_dict = dict(sorted(diff_dict.items()))
    # Итерация по ключам
    sum = 0
    for key in sorted_dict:
        sum += int(sorted_dict[key])*(pow(2, int(key)))
    if sum == 0:
        print('FULL MATCH')
    return sum, sorted_dict

In [74]:
best_adapters = ''
diff_dict = {}
coeff_dict_of_dicts = {}
coordinates_dict = {}

# iterating over every chinese adapter
# Берем только первые 96, потому что именно столько у нас в китайском наборе.
# Как только он зайдет в all_column_diff, он пробежится по ВСЕМ рошевским адаптерам,
# итерируя по каждому китайскому адаптеру по отдельности.
for adapter in adapters['china_nextseq'].iloc[:96]: 
     diff, local_dict = all_column_diff(adapter, adapters) 
     coeff_dict_of_dicts[adapter] = local_dict 
     diff_dict[adapter] = diff
     coordinates = [(x, adapters.columns[y]) for x, y in zip(*np.where(adapters.values == adapter))]  # [(3, 'C')]
     coordinates_dict[adapter] = int(coordinates[0][0])+1

#taking the Chinese ones who have the least (and most) matches with the Roche adapters
import heapq
diff_small_list = heapq.nsmallest(24, diff_dict, key=diff_dict.get)
for adapter in diff_small_list:
     print(adapter, coordinates_dict[adapter], diff_dict[adapter], coeff_dict_of_dicts[adapter])
diff_big_list = heapq.nlargest(24, diff_dict, key=diff_dict.get)
#for adapter in diff_big_list:
     #print(adapter, coordinates_dict[adapter], diff_dict[adapter], coeff_dict_of_dicts[adapter])



CCGTTAATCTTGGCTT 91 4356 {1: 4, 2: 21, 3: 49, 4: 56, 5: 39, 6: 19, 7: 4}
CTTGAGCATACTTACT 55 4622 {0: 2, 1: 4, 2: 15, 3: 55, 4: 49, 5: 44, 6: 16, 7: 7}
TGAGCCTCCCATATAA 51 4794 {1: 13, 2: 26, 3: 43, 4: 52, 5: 29, 6: 18, 7: 11}
ACAGAAGTAAGGCGGT 4 4932 {1: 4, 2: 27, 3: 34, 4: 64, 5: 34, 6: 22, 7: 6, 8: 1}
AGGACGCGTACCGACC 96 4983 {0: 1, 1: 11, 2: 20, 3: 38, 4: 54, 5: 42, 6: 19, 7: 5, 8: 2}
AGAAGACAGCTTATAT 34 5011 {0: 1, 1: 9, 2: 22, 3: 53, 4: 56, 5: 22, 6: 21, 7: 4, 8: 4}
GCAGAGGACAAGGCGA 40 5028 {0: 2, 1: 5, 2: 22, 3: 42, 4: 57, 5: 41, 6: 13, 7: 8, 8: 2}
AACAGATCAGTTAGCC 38 5070 {0: 2, 1: 14, 2: 16, 3: 44, 4: 53, 5: 34, 6: 22, 7: 4, 8: 3}
ATGAGGCATATGGCGG 84 5091 {0: 3, 1: 6, 2: 31, 3: 39, 4: 42, 5: 44, 6: 18, 7: 7, 8: 2}
AACGCCACTCCGCTGC 73 5199 {0: 1, 1: 11, 2: 24, 3: 43, 4: 36, 5: 48, 6: 21, 7: 6, 8: 2}
TAGAACTTCTTCCATT 43 5298 {0: 2, 1: 10, 2: 31, 3: 40, 4: 44, 5: 37, 6: 16, 7: 9, 8: 3}
TTAGACAACGAACAGG 68 5340 {1: 6, 2: 26, 3: 41, 4: 52, 5: 37, 6: 17, 7: 12, 8: 1}
GGTCGACCTCGTGCAC

In [53]:
print(sys.getsizeof(diff_dict))

3328


In [54]:
#works only for finding the biggest similarity (bad)
for i in range (3, 5):
    print(f'Adapters similar by {i} nucleotides:')
    for key, inner_dict in coeff_dict_of_dicts.items():
    # Check if key 10 is present in the inner dictionary
        if i in inner_dict:
            print(f'{key}: coordinate {coordinates_dict[key]}, coefficient {diff_dict[key]} {coeff_dict_of_dicts[key]}')

Adapters similar by 3 nucleotides:
TTGTGTCCTGCTTATC: coordinate 1, coefficient 6477 {0: 1, 1: 10, 2: 26, 3: 40, 4: 35, 5: 45, 6: 19, 7: 12, 8: 3, 9: 1}
CACCTCAGCCTAATAG: coordinate 2, coefficient 5862 {0: 2, 1: 6, 2: 30, 3: 42, 4: 45, 5: 32, 6: 17, 7: 16, 8: 2}
GGTACGTAGTACGCCA: coordinate 3, coefficient 7263 {0: 1, 1: 9, 2: 33, 3: 39, 4: 41, 5: 34, 6: 17, 7: 11, 8: 4, 9: 3}
ACAGAAGTAAGGCGGT: coordinate 4, coefficient 4932 {1: 4, 2: 27, 3: 34, 4: 64, 5: 34, 6: 22, 7: 6, 8: 1}
TAAGGAGTAATCGGCA: coordinate 5, coefficient 5358 {1: 9, 2: 27, 3: 42, 4: 46, 5: 38, 6: 20, 7: 7, 8: 3}
ACTTCCAGGCCACCTG: coordinate 6, coefficient 8191 {0: 3, 1: 2, 2: 24, 3: 37, 4: 45, 5: 37, 6: 26, 7: 11, 8: 5, 9: 1, 10: 1}
GTCCATCACGAGTAAC: coordinate 7, coefficient 5478 {0: 2, 1: 4, 2: 19, 3: 46, 4: 48, 5: 43, 6: 19, 7: 9, 8: 2}
CGGATGTCTTGTATGT: coordinate 8, coefficient 10992 {0: 6, 1: 25, 2: 30, 3: 32, 4: 36, 5: 26, 6: 15, 7: 12, 8: 6, 10: 3, 11: 1}
TGCAAGCCTCCACAGT: coordinate 9, coefficient 11724 {0: 4, 1

In [36]:
print('CHINA_MISEQ')

CHINA_MISEQ


In [73]:
print(7)
for key, inner_dict in coeff_dict_of_dicts.items():
    max_key = max(inner_dict, default=0)  # Find the maximum key = matches between the Chinese adapters and Roche
    if max_key == 8:  # Check if the maximum key is within the specified range
        #coefficient {diff_dict[key]}
        print(f' {coordinates_dict[key]}: {key}:  {coeff_dict_of_dicts[key]}')

7
 2: CACCTCAGCCTAATAG:  {0: 2, 1: 6, 2: 30, 3: 42, 4: 45, 5: 32, 6: 17, 7: 16, 8: 2}
 4: ACAGAAGTAAGGCGGT:  {1: 4, 2: 27, 3: 34, 4: 64, 5: 34, 6: 22, 7: 6, 8: 1}
 5: TAAGGAGTAATCGGCA:  {1: 9, 2: 27, 3: 42, 4: 46, 5: 38, 6: 20, 7: 7, 8: 3}
 7: GTCCATCACGAGTAAC:  {0: 2, 1: 4, 2: 19, 3: 46, 4: 48, 5: 43, 6: 19, 7: 9, 8: 2}
 11: GTATGTAGATAGTGCC:  {0: 5, 1: 9, 2: 19, 3: 40, 4: 44, 5: 34, 6: 18, 7: 13, 8: 10}
 16: CTAGTAAGATCCGTCG:  {0: 3, 1: 6, 2: 21, 3: 55, 4: 34, 5: 37, 6: 22, 7: 10, 8: 4}
 28: AAGGTGTGATTGCAGC:  {1: 12, 2: 20, 3: 46, 4: 44, 5: 45, 6: 13, 7: 9, 8: 3}
 32: TCGCTCCACAATAGCG:  {1: 17, 2: 21, 3: 38, 4: 41, 5: 29, 6: 26, 7: 12, 8: 8}
 34: AGAAGACAGCTTATAT:  {0: 1, 1: 9, 2: 22, 3: 53, 4: 56, 5: 22, 6: 21, 7: 4, 8: 4}
 38: AACAGATCAGTTAGCC:  {0: 2, 1: 14, 2: 16, 3: 44, 4: 53, 5: 34, 6: 22, 7: 4, 8: 3}
 40: GCAGAGGACAAGGCGA:  {0: 2, 1: 5, 2: 22, 3: 42, 4: 57, 5: 41, 6: 13, 7: 8, 8: 2}
 41: GGTCGACCTCGTGCAC:  {0: 1, 1: 11, 2: 27, 3: 45, 4: 42, 5: 33, 6: 23, 7: 7, 8: 3}
 43: TAGA

In [67]:
print(8)
for key, inner_dict in coeff_dict_of_dicts.items():
    max_key = max(inner_dict, default=0)  # Find the maximum key
    if max_key == 11:  # Check if the maximum key is within the specified range
        #coefficient {diff_dict[key]}
        print(f' {coordinates_dict[key]}: {key}: {coeff_dict_of_dicts[key]}')

8
 8: CGGATGTCTTGTATGT:  coefficient 10992 {0: 6, 1: 25, 2: 30, 3: 32, 4: 36, 5: 26, 6: 15, 7: 12, 8: 6, 10: 3, 11: 1}
 9: TGCAAGCCTCCACAGT:  coefficient 11724 {0: 4, 1: 22, 2: 37, 3: 37, 4: 32, 5: 19, 6: 20, 7: 11, 8: 3, 9: 5, 11: 2}
 23: AAGCTAGACTATTCAG:  coefficient 10374 {1: 13, 2: 23, 3: 32, 4: 45, 5: 34, 6: 24, 7: 12, 8: 4, 9: 4, 11: 1}
 46: GAGGCTAGCATTGTGC:  coefficient 9846 {0: 4, 1: 15, 2: 29, 3: 36, 4: 36, 5: 32, 6: 18, 7: 12, 8: 8, 10: 1, 11: 1}
 71: ATCTTCACTATTCCTG:  coefficient 13046 {0: 4, 1: 11, 2: 25, 3: 31, 4: 38, 5: 31, 6: 21, 7: 14, 8: 11, 9: 4, 10: 1, 11: 1}
 81: TCATCCGCATGCTGCA:  coefficient 12953 {0: 3, 1: 15, 2: 28, 3: 31, 4: 37, 5: 28, 6: 19, 7: 13, 8: 12, 9: 4, 10: 1, 11: 1}
 87: CCAGGACAACTGATCT:  coefficient 11439 {0: 7, 1: 18, 2: 33, 3: 40, 4: 38, 5: 15, 6: 22, 7: 8, 8: 5, 9: 2, 10: 3, 11: 1}


In [56]:
best_adapters = ''
diff_dict = {}
coeff_dict_of_dicts = {}
coordinates_dict = {}

#iterating over every chinese adapter
for adapter in adapters['china_no_miseq']:
     diff, local_dict = all_column_diff(adapter, adapters)
     coeff_dict_of_dicts[adapter] = local_dict
     diff_dict[adapter] = diff
     coordinates = [(x, adapters.columns[y]) for x, y in zip(*np.where(adapters.values == adapter))] 
     coordinates_dict[adapter] = int(coordinates[0][0])+1

#taking the Chinese ones who have the least (and most) matches with the Roche adapters
import heapq
diff_small_list = heapq.nsmallest(24, diff_dict, key=diff_dict.get)
for adapter in diff_small_list:
     print(adapter, coordinates_dict[adapter], diff_dict[adapter], coeff_dict_of_dicts[adapter])
diff_big_list = heapq.nlargest(24, diff_dict, key=diff_dict.get)
#for adapter in diff_big_list:
     #print(adapter, coordinates_dict[adapter], diff_dict[adapter], coeff_dict_of_dicts[adapter])



AGACCAACAGTGGCGG 56 2263 {0: 3, 1: 10, 2: 18, 3: 15, 4: 24, 5: 14, 6: 9, 7: 1, 8: 2}
CTGTTGGTTTCCGGCG 50 2268 {0: 2, 1: 9, 2: 12, 3: 21, 4: 25, 5: 13, 6: 11, 7: 2, 8: 1}
GGAGATAGTGTGGCGC 64 2417 {0: 1, 1: 8, 2: 14, 3: 23, 4: 23, 5: 14, 6: 9, 7: 2, 8: 2}
TTACTTGTAATTGAAG 75 2469 {0: 1, 1: 8, 2: 17, 3: 22, 4: 16, 5: 17, 6: 12, 7: 1, 8: 2}
CTAGTAAGATCCGTCG 16 2488 {1: 10, 2: 15, 3: 13, 4: 24, 5: 18, 6: 13, 7: 2, 8: 1}
TGAAGGCAAGCAGGCC 72 2520 {1: 8, 2: 20, 3: 19, 4: 20, 5: 17, 6: 8, 7: 3, 9: 1}
ATGAGGCATATGGCGG 84 2546 {1: 9, 2: 18, 3: 21, 4: 17, 5: 15, 6: 12, 7: 2, 8: 2}
TCAGTAACTTGCGGCA 79 2584 {1: 6, 2: 19, 3: 24, 4: 22, 5: 11, 6: 7, 7: 5, 8: 2}
ACAGAAGTAAGGCGGT 4 2598 {0: 2, 1: 10, 2: 12, 3: 22, 4: 23, 5: 12, 6: 9, 7: 4, 8: 2}
GATCAGATGGACGGTG 88 2599 {0: 1, 1: 5, 2: 17, 3: 23, 4: 18, 5: 14, 6: 13, 7: 4, 8: 1}
ATCGCTGGGACGAGGA 44 2613 {0: 1, 1: 8, 2: 13, 3: 24, 4: 17, 5: 13, 6: 14, 7: 6}
CCATTGAAAGAATTCG 42 2638 {1: 3, 2: 18, 3: 24, 4: 20, 5: 16, 6: 10, 7: 3, 8: 2}
GCAGAGGACAAGGCGA 40

In [61]:
for key, inner_dict in coeff_dict_of_dicts.items():
    max_key = max(inner_dict, default=0)  # Find the maximum key
    if max_key <= 7:  # Check if the maximum key is within the specified range
        print(f' {coordinates_dict[key]}: {key}: {diff_dict[key]} {coeff_dict_of_dicts[key]}')

 17: GGACCGTAATTCGCCT: 2818 {0: 2, 1: 8, 2: 12, 3: 18, 4: 19, 5: 18, 6: 11, 7: 8}
 44: ATCGCTGGGACGAGGA: 2613 {0: 1, 1: 8, 2: 13, 3: 24, 4: 17, 5: 13, 6: 14, 7: 6}
 77: CGTAGTGGAACACCGT: 2685 {0: 1, 1: 6, 2: 10, 3: 17, 4: 24, 5: 22, 6: 10, 7: 6}


In [62]:
for key, inner_dict in coeff_dict_of_dicts.items():
    max_key = max(inner_dict, default=0)  # Find the maximum key
    if max_key == 8:  # Check if the maximum key is within the specified range
        print(f' {coordinates_dict[key]}: {key}:  {diff_dict[key]} {coeff_dict_of_dicts[key]}')

 4: ACAGAAGTAAGGCGGT:  2598 {0: 2, 1: 10, 2: 12, 3: 22, 4: 23, 5: 12, 6: 9, 7: 4, 8: 2}
 5: TAAGGAGTAATCGGCA:  2889 {0: 3, 1: 7, 2: 16, 3: 23, 4: 20, 5: 10, 6: 7, 7: 8, 8: 2}
 10: AAGCTCGTCGTTATTA:  3672 {1: 4, 2: 14, 3: 15, 4: 22, 5: 18, 6: 12, 7: 8, 8: 3}
 11: GTATGTAGATAGTGCC:  2719 {0: 1, 1: 3, 2: 20, 3: 25, 4: 16, 5: 10, 6: 17, 7: 2, 8: 2}
 12: CCTGCATAGAGCGCAG:  3297 {0: 1, 1: 6, 2: 13, 3: 20, 4: 24, 5: 10, 6: 13, 7: 6, 8: 3}
 13: ACCTGGTCGAAGCAGA:  2913 {0: 1, 1: 6, 2: 15, 3: 15, 4: 28, 5: 15, 6: 10, 7: 3, 8: 3}
 16: CTAGTAAGATCCGTCG:  2488 {1: 10, 2: 15, 3: 13, 4: 24, 5: 18, 6: 13, 7: 2, 8: 1}
 18: CTGTACGCCAAGCGGC:  3119 {0: 1, 1: 9, 2: 11, 3: 20, 4: 23, 5: 15, 6: 8, 7: 6, 8: 3}
 21: GTCTGGAGACGAATCT:  2856 {1: 6, 2: 19, 3: 18, 4: 18, 5: 19, 6: 9, 7: 5, 8: 2}
 24: TCAACTTCGATCCGGA:  2811 {0: 1, 1: 7, 2: 11, 3: 24, 4: 22, 5: 15, 6: 9, 7: 5, 8: 2}
 26: GGCTAAGAGCATTGAT:  3757 {0: 3, 1: 5, 2: 12, 3: 18, 4: 16, 5: 21, 6: 7, 7: 11, 8: 3}
 27: CTACCTCTCGGCATTA:  4039 {0: 5, 1: 3, 2: